In [94]:
import pandas as pd
df = pd.read_csv('datos_full.csv')

In [95]:
#Eliminar las que tienen habitaciones y baños null en la misma fila
filas_nulas = df[(df['Habitaciones'].isnull()) & (df['Baños'].isnull())]

print(filas_nulas.count())
df = df.drop(filas_nulas.index)

id_inmueble               112
Superficie construida     112
Superficie útil            50
Habitaciones                0
Baños                       0
Terraza                    23
Balcón                     28
Plaza de garaje            17
Estado                    100
Armarios empotrados        59
Calefacción                49
Orientación                64
Calefacción individual     31
Consumo                    27
Emisiones                  26
Barrio                     82
Distrito                  111
Precio                    112
Calefacción central         1
3                          31
dtype: int64


In [96]:
#Unir columnas para centrar datos similares

df['Habitaciones'] = df['Habitaciones'].astype(str)
df['Baños'] = df['Baños'].astype(str)

df['Habitaciones'] = df['Habitaciones'].str.extract('(\d+)').astype(float)

df['Baños'] = df['Baños'].str.extract('(\d+)').astype(float)

def combine_terrazas_salon(row):
    terraza = str(row['Terraza']) if pd.notnull(row['Terraza']) else ''
    salon = str(row['Balcón']) if pd.notnull(row['Balcón']) else ''
    if terraza == salon:
        return terraza
    return ' '.join(filter(None, [terraza, salon]))

df['Terraza_Balcon'] = df.apply(combine_terrazas_salon, axis=1)

def combine_calefaccion(row):
    calefaccion = str(row['Calefacción']) if pd.notnull(row['Calefacción']) else ''
    calefaccion_central = str(row['Calefacción central']) if pd.notnull(row['Calefacción central']) else ''
    calefaccion_individual = str(row['Calefacción individual']) if pd.notnull(row['Calefacción individual']) else ''
    if calefaccion and calefaccion_central:
        return ' '.join(filter(None, [calefaccion, calefaccion_central]))
    elif calefaccion:
        return calefaccion
    elif calefaccion_central:
        return calefaccion_central
    elif calefaccion_individual:
        return calefaccion_individual
    return ''

df['Calefacción'] = df.apply(combine_calefaccion, axis=1)

In [97]:
#Rellenar los nulos con 0, lo cual quiere decir que no tienen en el caso del garaje y los armarios empotrados
#En el caso del estado, el 0 es que son nuevos

df['Plaza de garaje'] = df['Plaza de garaje'].fillna(0)
df['Estado'] = df['Estado'].fillna(0)
df['Armarios empotrados'] = df['Armarios empotrados'].fillna(0)

In [98]:
#Reemplazar valores nulos en 'Baños' segun el numero de habitaciones
media_baños = df.groupby('Habitaciones')['Baños'].mean().round()
print(media_baños)

def rellenar_baños(row):
    if pd.isnull(row['Baños']):
        return media_baños.get(row['Habitaciones'], row['Baños']) 
    return row['Baños']

df['Baños'] = df.apply(lambda row: rellenar_baños(row), axis=1)

Habitaciones
2.0     2.0
3.0     2.0
4.0     2.0
5.0     3.0
6.0     3.0
7.0     3.0
8.0     4.0
9.0     3.0
10.0    2.0
Name: Baños, dtype: float64


In [99]:
import re

#Añadir precio plaza de garaje al precio final de la vivienda
df['Plaza de garaje'] = df['Plaza de garaje'].fillna('')

def extract_price(value):
    if isinstance(value, str) and '€' in value:
        match = re.search(r'(\d+(?:\.\d{1,3})?)', value)
        return float(match.group().replace('.', '')) if match else 0
    else:
        return 0

df['Precio garaje'] = df['Plaza de garaje'].apply(extract_price)
df['Precio'] = df['Precio'] + df['Precio garaje']

In [100]:
#Eliminacion de columnas
to_drop = ['id_inmueble', 'Superficie útil', 'Terraza', 'Balcón', '3', 'Calefacción central', 'Calefacción individual', 'Consumo', 'Emisiones','Barrio','Orientación', 'Precio garaje', 'Plaza de garaje']
df.drop(to_drop, axis = 1, inplace = True)

In [101]:
df.dropna(inplace=True)

In [102]:
#Columans a tipo string
df[['Estado', 'Armarios empotrados', 'Calefacción', 'Distrito', 'Terraza_Balcon']] = \
    df[['Estado', 'Armarios empotrados', 'Calefacción', 'Distrito', 'Terraza_Balcon']].astype(str)


In [103]:
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

oe = OrdinalEncoder(handle_unknown='error', dtype=np.int8)

# Almacenar el mapeo original de los distritos
distritos_unicos = df['Distrito'].unique()
distritos_mapeo = {distrito: idx for idx, distrito in enumerate(distritos_unicos)}

# Transformar las columnas categóricas a valores numéricos
df[['Estado', 'Armarios empotrados', 'Calefacción', 'Distrito', 'Terraza_Balcon']] = \
  oe.fit_transform(df[['Estado', 'Armarios empotrados', 'Calefacción', 'Distrito', 'Terraza_Balcon']])

# Crear un DataFrame para ver el mapeo original
distritos_df = pd.DataFrame(list(distritos_mapeo.items()), columns=['Distrito Original', 'Distrito Codificado'])
print(distritos_df)

            Distrito Original  Distrito Codificado
0         Distrito L'Eixample                    0
1     Distrito Camins al Grau                    1
2    Distrito Quatre Carreres                    2
3       Distrito Ciutat Vella                    3
4    Distrito El Pla del Real                    4
5           Distrito Campanar                    5
6          Distrito Extramurs                    6
7          Distrito Benicalap                    7
8         Distrito Benimaclet                    8
9        Distrito L'Olivereta                    9
10  Distrito Poblats Marítims                   10
11         Distrito La Saïdia                   11
12           Distrito Patraix                   12
13           Distrito Algirós                   13
14             Distrito Jesús                   14
15          Distrito Rascanya                   15


In [104]:
#Columans a tipo float
df = df.astype('float64')

In [105]:
df.to_csv('datos_ML.csv', index = False)